<a href="https://colab.research.google.com/github/Sixie0513/ML100Days/blob/main/krono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install logbook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from ctypes import sizeof
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
import shutil
import pandas as pd
from logbook import Logger
import logbook
from zipfile import ZipFile
import time
import os

log = Logger('binance_data.py', level=logbook.INFO)

columns = ['ID', 'price', 'size_in_quote', 'trade_time_in_millisecond', 'is_the_buyer_market_maker', 'ignore']


def download_spot_klines(symbol, way, date: datetime, path):
    url = 'https://data.binance.vision/data/spot/daily/%s/%s/%s-%s-%s-%02d-%02d.zip' % (way, symbol, symbol, way, date.year, date.month, date.day)
    log.info(url)

    with requests.get(url, stream=True) as r:
        with open(path, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return path


def download_spot_klines_range(symbol, way, start: datetime, end: datetime, dir=''):

    if len(dir) > 0 and not dir.endswith('/'):
        dir += '/'

    os.makedirs('%s%s/%s' % (dir, way, symbol), exist_ok=True)

    while True:

        path = '%s%s/%s/%s-%s-%s-%02d-%02d.zip' % (dir, way, symbol, symbol, way, start.year, start.month, start.day)
        csv_file = '%s-%s-%s-%02d-%02d.csv' % (symbol, way, start.year, start.month, start.day)

        download_spot_klines(symbol, way, start, path)

        zf = ZipFile(path)
        file = zf.open(csv_file)
        df = pd.read_csv(file, header=None, names=columns)

        sma1 = df.size_in_quote.rolling(20).mean()
        sma2 = df.size_in_quote.rolling(60).mean()
        
        df.size_in_quote.plot()
        sma1.plot()
        sma2.plot()

        file.close()
        zf.close()

        start = start + relativedelta(days=+1)

        if start > end:
            break

        time.sleep(2)

if __name__ == '__main__':
    print("0")
    download_spot_klines_range('BTCUSDT', 'trades', start=datetime(2022, 7, 1), end=datetime(2022, 7, 2))

0
1430939793
1430939794
1430939795
1430939796
1430939797
1430939798
1430939799
1430939800
1430939801
1430939802
1430939803
1430939804
1430939805
1430939806
1430939807
1430939808
1430939809
1430939810
1430939811
1430939812
1430939813
1430939814
1430939815
1430939816
1430939817
1430939818
1430939819
1430939820
1430939821
1430939822
1430939823
1430939824
1430939825
1430939826
1430939827
1430939828
1430939829
1430939830
1430939831
1430939832
1430939833
1430939834
1430939835
1430939836
1430939837
1430939838
1430939839
1430939840
1430939841
1430939842
1430939843
1430939844
1430939845
1430939846
1430939847
1430939848
1430939849
1430939850
1430939851
1430939852
1430939853
1430939854
1430939855
1430939856
1430939857
1430939858
1430939859
1430939860
1430939861
1430939862
1430939863
1430939864
1430939865
1430939866
1430939867
1430939868
1430939869
1430939870
1430939871
1430939872
1430939873
1430939874
1430939875
1430939876
1430939877
1430939878
1430939879
1430939880
1430939881
1430939882
14309398

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


串流輸出內容已截斷至最後 5000 行。
1431144052
1431144053
1431144054
1431144055
1431144056
1431144057
1431144058
1431144059
1431144060
1431144061
1431144062
1431144063
1431144064
1431144065
1431144066
1431144067
1431144068
1431144069
1431144070
1431144071
1431144072
1431144073
1431144074
1431144075
1431144076
1431144077
1431144078
1431144079
1431144080
1431144081
1431144082
1431144083
1431144084
1431144085
1431144086
1431144087
1431144088
1431144089
1431144090
1431144091
1431144092
1431144093
1431144094
1431144095
1431144096
1431144097
1431144098
1431144099
1431144100
1431144101
1431144102
1431144103
1431144104
1431144105
1431144106
1431144107
1431144108
1431144109
1431144110
1431144111
1431144112
1431144113
1431144114
1431144115
1431144116
1431144117
1431144118
1431144119
1431144120
1431144121
1431144122
1431144123
1431144124
1431144125
1431144126
1431144127
1431144128
1431144129
1431144130
1431144131
1431144132
1431144133
1431144134
1431144135
1431144136
1431144137
1431144138
1431144139
1431144140
